In [1]:
import pandas as pd
import numpy as np
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
import pprint
from bs4 import BeautifulSoup

In [2]:
# import apartment info from mongodb
db = client['apartments']
table = db['main_pages']

In [3]:
table.count_documents({})

586

In [4]:
apartment_collection = table.find()

In [39]:
#create new dataframe with just headers

# for each apartment in collection parse html and 
# find the following info in collection 
# [name, address, price or price range, area, and Walkscore] 
# and add to Pandas DF
for apartment in apartment_collection:
    soup = BeautifulSoup(apartment['html'], 'html.parser')
    
    listing_id = apartment['listing_id']
    property_name = soup.find('h1', 'propertyName').text.strip()
    address_ = soup.find('div', 'propertyAddress').find('h2').text
    address = ' '.join(address_.split())
    
    # iterate through each unit and find mean price and area
    max_rents = []
    for unit in soup.find('table', 'availabilityTable').find('tbody').find_all('tr', 'rentalGridRow'):
        availability = unit.find('td', 'available').text.strip()
        # only look at max rents for simplicity
        unit_max_rent = unit['data-maxrent']
        if (availability == 'Available Now') and (unit_max_rent != ''):
            max_rents.append(int(unit_max_rent))
        else:
            break
            
    mean_rent = sum(max_rents) / len(max_rents)
    print(address)
    print(max_rents)
    print(mean_rent)
    mean_area = 0
    break
print(address)

3701 Quick Hill Rd, Austin, TX 78728
[2765, 2149, 2765, 2207, 2207, 3149, 2337, 2447, 3149, 4204, 3589, 3655, 4176, 4204, 3668, 3668, 4189, 3750, 3731, 4189]
3309.9
3701 Quick Hill Rd, Austin, TX 78728


In [6]:
# Create column for cost/SF

# Create column for some metric comparing cost/SF and Walkscore. 
# Maybe it's the ((cost/SF) / Walkscore)

In [7]:
address

'2021-2025 Guadalupe St, Austin, TX 78705'

In [8]:
lst = []
sum(lst) / len(lst)

ZeroDivisionError: division by zero